In [ ]:
import json
import os
import re
import traceback

import math
import scipy
import scipy.stats
import numpy as np

import pyzipcode
import hashlib

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown
from IPython.display import HTML
from tqdm import tqdm

import sqlite3
import pandas as pd
import nltk

import itertools
import collections
import functools
import collections


import statsmodels.stats.multitest as multitest
import statsmodels.api as sm



In [ ]:
sns.set_style(style="whitegrid")

In [ ]:
yc_reviews = pd.read_pickle("../data/pickles/yelpchi_reviews.pkl")
yc_updated_reviews = pd.read_pickle("../data/pickles/yelpchi_updated_reviews.pkl")
yc_businesses = pd.read_pickle("../data/pickles/yelpchi_business_data_with_distances.pkl")
business_data = pd.read_pickle("../data/pickles/yelpchi_businesses.pkl")

In [ ]:
len(yc_reviews.businessID.unique())

In [ ]:
business_data.yc_type.value_counts()

## Question: is the data hovercard ID a unique identifier per user?

In [ ]:
matches = yc_reviews[yc_reviews.matchID.notnull()].apply(lambda row: pd.Series([row.matchID, row.reviewerID, yc_updated_reviews.loc[row.matchID].reviewerID]), axis=1)
matches

In [ ]:
matches[matches[2].isnull()][1].apply(lambda revId: pd.Series([revId, len(yc_reviews[yc_reviews.reviewerID == revId])])).sort_values([1])

In [ ]:
yc_reviews[yc_reviews.matchID.notnull() & (yc_reviews.reviewerID == "QZrC4uhuZCwt9uXeg9uwnA")].matchID.apply(lambda x: yc_updated_reviews.loc[x])

#### Answer -- YES! Although it doesn't match the user ID

## How many of the reviews for which we've crawled the restaurants do we have an exact match?

In [ ]:
yc_reviews_we_have = yc_reviews[yc_reviews.businessID.isin(yc_updated_reviews.businessID.unique())]

In [ ]:
print(yc_reviews_we_have.groupby("flagged").size())
print(yc_reviews.groupby("flagged").size())

In [ ]:
yc_updated_reviews["year"] = yc_updated_reviews.date.apply(lambda d: d.year)

In [ ]:
yc_reviews.groupby("flagged").size().loc["Y"] / len(yc_reviews),yc_updated_reviews.groupby("flagged").size().loc["Y"] / len(yc_updated_reviews),yc_updated_reviews[yc_updated_reviews.year<=2012].groupby("flagged").size().loc["Y"] / len(yc_updated_reviews)

In [ ]:
len(yc_reviews_we_have), len(yc_reviews)

In [ ]:
len(yc_reviews_we_have[yc_reviews_we_have.matchID.isnull()]), len(yc_reviews_we_have)

#### Not enough

# Question: Can we match filtered and recommended review's users?

In [ ]:

reviews_with_user_id = yc_updated_reviews[yc_updated_reviews.user_page_url.notnull()]

name_matches = collections.defaultdict(set)
name_matches_exact = collections.defaultdict(set)

out_of = len(yc_updated_reviews[yc_updated_reviews.user_page_url.isnull()])

for rowid, row in tqdm(yc_updated_reviews[yc_updated_reviews.user_page_url.isnull()].iterrows(), total=out_of):
    user_name = row.user_name
    user_friends = row.user_friends
    user_photos = row.user_photos
    user_reviews = row.user_review_count
    user_location = row.user_location
    m = reviews_with_user_id[
        (reviews_with_user_id.user_name == user_name) &
        (reviews_with_user_id.user_location == user_location) &
        ((reviews_with_user_id.user_friends - user_friends).abs() <= 1) &
        ((reviews_with_user_id.user_photos - user_photos).abs() <= 1) &
        ((reviews_with_user_id.user_review_count - user_reviews).abs() <= 1)
    ]
    m_exact = m[
        (m.user_name == user_name) &
        (m.user_location == user_location) &
        ((m.user_friends - user_friends).abs() == 0) &
        ((m.user_photos - user_photos).abs() == 0) &
        ((m.user_review_count - user_reviews).abs() == 0)
    ]
    if len(m) > 0:
        name_matches[row.data_hovercard_id].update(m.reviewerID)
        name_matches_exact[row.data_hovercard_id].update(m_exact.reviewerID)
        #if len(name_matches) < 10:
        #    display(pd.concat([yc_updated_reviews[yc_updated_reviews.data_hovercard_id == row.data_hovercard_id],m]))
        #    display(Markdown('<hr style="height:8px;border-width:0;color:gray;background-color:red">'))
    else:
        name_matches[row.data_hovercard_id].update([])
        name_matches_exact[row.data_hovercard_id].update([])
            
print(f"Found {len(name_matches)} approximate and {len(name_matches_exact)} exact matches.")

In [ ]:
num_matches = pd.Series([len(l) for l in name_matches.values()])
num_matches_exact = pd.Series([len(name_matches_exact[l]) for l in name_matches.keys()])
bins=list(set(num_matches) | set(num_matches_exact))
sns.distplot(num_matches,bins=bins,label="Approximate matches",kde=False)
sns.distplot(num_matches_exact,bins=bins,label="Exact matches",kde=False)
print(num_matches.value_counts())
print(num_matches_exact.value_counts())

In [ ]:
num_matches_exact.value_counts()[1]

In [ ]:
updated_recommended_reviewers = len(yc_updated_reviews[yc_updated_reviews.user_page_url.notnull()].reviewerID.unique())
updated_filtered_reviewers = len(yc_updated_reviews[yc_updated_reviews.user_page_url.isnull()].data_hovercard_id.unique())

original_recommended_reviewers = len(yc_reviews[yc_reviews.flagged == "N"].reviewerID.unique())
original_filtered_reviewers = len(yc_reviews[yc_reviews.flagged == "Y"].reviewerID.unique())

updated_dual_reviewers = num_matches_exact.value_counts()[1]#yc_updated_reviews.groupby("reviewerID").apply(lambda df: bool(len(df[df.flagged == "N"]) != 0 and len(df[df.flagged == "Y"]))).value_counts()[True]
original_dual_reviewers = yc_reviews.groupby("reviewerID").apply(lambda df: bool(len(df[df.flagged == "N"]) != 0 and len(df[df.flagged == "Y"]))).value_counts()[True]

print(f"""Filtered reviewers (original/updated): {original_filtered_reviewers}/{updated_filtered_reviewers}
Recommended reviewers (original/updated): {original_recommended_reviewers}/{updated_recommended_reviewers}
Dual reviewers (original/updated): {original_dual_reviewers}/{updated_dual_reviewers}
Total reviewers (original/updated): {len(yc_reviews.reviewerID.unique())}/{len(yc_updated_reviews.reviewerID.unique())} to {len(yc_updated_reviews.reviewerID.unique()) + len(yc_updated_reviews.data_hovercard_id.unique())}
""")

## Looking at unmatched reviews

In [ ]:
unmatched_reviews = yc_reviews_we_have[yc_reviews_we_have.matchID.isnull()]
len(unmatched_reviews[unmatched_reviews.flagged == "Y"]) / len(unmatched_reviews)

# Question: How much churn is there between filtered and recommended?

In [ ]:
yc_matched = yc_reviews_we_have[yc_reviews_we_have.matchID.notnull()]

In [ ]:
len(yc_matched), len(yc_updated_reviews) - len(yc_matched), len(unmatched_reviews)

In [ ]:
#Capture percentage
len(yc_matched)/len(yc_reviews_we_have)

In [ ]:
#Filtered percentage
len(yc_matched[yc_matched.flagged == "Y"]) / len(yc_matched)

In [ ]:
#Different approach to this
rows = []
for rowid, row in tqdm(yc_matched.iterrows(),total=len(yc_matched)):
    f1 = row.flagged
    f2 = yc_updated_reviews.loc[row.matchID,].flagged
    elite = yc_updated_reviews.loc[row.matchID,].elite == True
    d = row.date
    #df_matches = df_matches.append([d,f1,f2,elite])
    rows.append([rowid,d,f1,f2,elite])
    

df_matches = pd.DataFrame(rows,columns=["rowid","date","f1","f2","elite"])

In [ ]:
df_matches["year"] = df_matches.date.apply(lambda d: d.year)

In [ ]:
def get_fisher_table(df,year):
    vals = []
    for f1,f2 in (("N","N"),("N","Y"),("Y","N"),("Y","Y")):
        try:
            v = df.loc[(year,f1,f2),"ct"]
        except KeyError:
            v = 0
        vals.append(v)
    return np.array(vals).reshape(2,2)

In [ ]:
param_ar = []
for year, df in df_matches.groupby(["year","f1","f2"]).size().rename("ct").reset_index().groupby("year"):
    #display(df)
    df = df.set_index(["year","f1","f2"])
    df["perc"] = df.ct / df.ct.sum()
    #df["exp_perc"] = overall_percs[df.index.droplevel("year")].to_numpy()
    print(year)
    display(df)
    
    try:
        p = df.loc[(year,"Y"),"ct"].sum() / df.ct.sum()
    except:
        p = 0
    a = df.loc[(year,"N","Y"),"ct"] / df.loc[(year,"N"),"ct"].sum()
    try:
        b = df.loc[(year,"Y","N"),"ct"] / df.loc[(year,"Y"),"ct"].sum()
    except:
        b = None
    param_ar.append((year,"% Not Recommended (2012)",p))
    param_ar.append((year,"% Recommended -> Not Recommended",a))
    param_ar.append((year,"% Not Recommended -> Recommended",b))
    
    obs = df.ct.to_numpy()
    #exp = (overall_percs[df.index.droplevel("year")] * df.ct.sum()).to_numpy()
    print(obs)
    #print(exp)
    obs_fisher = get_fisher_table(df,year)
    
    #display(scipy.stats.chisquare(obs,exp))
    display(scipy.stats.fisher_exact(obs_fisher))

In [ ]:
df_params = pd.DataFrame(param_ar, columns=["year","param","value"])
fig = sns.lineplot(x="year",y="value",data=df_params,hue="param")
plt.ylim(0,1)
plt.ylabel("Percentage of reviews")
plt.xlabel("Review year")


fig.get_figure().savefig("../graphs/filtering_changes.pdf")

In [ ]:

flagged_yy = 0
flagged_nn = 0
flagged_yn = 0
flagged_ny = 0
elite_yy = 0
elite_nn = 0
elite_yn = 0
elite_ny = 0

flag_change = None
for rowid, row in tqdm(yc_matched.iterrows(),total=len(yc_matched)):
    f1 = row.flagged
    f2 = yc_updated_reviews.loc[row.matchID,].flagged
    elite = yc_updated_reviews.loc[row.matchID,].elite == True
    if f1 == "Y" and f2 == "Y":
        flagged_yy += 1
        if elite:
            elite_yy += 1
    elif f1 == "N" and f2 == "Y":
        flagged_ny += 1
        if elite:
            elite_ny += 1
    elif f1 == "Y" and f2 == "N":
        flagged_yn += 1
        if elite:
            elite_yn += 1
    elif f1 == "N" and f2 == "N":
        flagged_nn += 1
        if elite:
            elite_nn += 1
    else:
        print(f1,f2)
        break

In [ ]:
data = pd.DataFrame([(flagged_nn,"n","n"),(flagged_yn,"y","n"),(flagged_ny,"n","y"),(flagged_yy,"y","y")],columns=["Count","FlaggedOriginal","FlaggedNew"])

In [ ]:
data

In [ ]:
print(flagged_yy,flagged_nn,flagged_yn,flagged_ny)

## Why do we see so much churn?

#### Is the churn from people becoming elite?

In [ ]:
len(yc_updated_reviews[yc_updated_reviews.elite == True]), len(yc_updated_reviews), len(yc_updated_reviews[yc_updated_reviews.elite == True]) / len(yc_updated_reviews)

In [ ]:
print(elite_yy,elite_nn,elite_yn,elite_ny)
print(elite_yy/flagged_yy,elite_nn/flagged_nn,elite_yn/flagged_yn,elite_ny/flagged_ny)

#### No -- clearly not

# Do Elites disproportionately influence rating?

In [ ]:
average_rating = yc_updated_reviews[yc_updated_reviews.flagged == "N"].groupby("businessID").rating.mean().rename("Average rating")
elite_rating = yc_updated_reviews[(yc_updated_reviews.elite == True) & (yc_updated_reviews.flagged == "N")].groupby("businessID").rating.mean().rename("Elite rating")
actual_rating = business_data.groupby("id").rating.mean().rename("Actual rating")

In [ ]:
ratings = pd.concat([average_rating,actual_rating,elite_rating],axis=1)
ratings

In [ ]:
ratings["average_diff"] = (ratings["Average rating"] - ratings["Actual rating"]).abs()
ratings["elite_diff"] = (ratings["Elite rating"] - ratings["Actual rating"]).abs()

In [ ]:
ratings[ratings["Average rating"].notnull()]

#### No

In [ ]:
print(",".join(list(yc_businesses)))

In [ ]:
df = ratings[ratings["Average rating"].notnull() & ratings["Elite rating"].notnull()]

scipy.stats.pearsonr(df["Elite rating"] - df["Average rating"], yc_businesses.loc[df.index].rating)

In [ ]:
fig = sns.scatterplot(x=df["Average rating"],y=df["Elite rating"] - df["Average rating"])
plt.ylabel("Elite rating - average rating")

In [ ]:
#df[df["Average rating"].notna() & df["Filtered rating"].notna()].sort_values("Average rating")

In [ ]:



average_rating_start = yc_reviews.groupby("businessID").apply(lambda df: df[df.flagged == "N"].rating.mean()).rename("Recommended rating")
filtered_rating_start = yc_reviews.groupby("businessID").apply(lambda df: df[df.flagged == "Y"].rating.mean()).rename("Filtered rating")
df = pd.concat([average_rating_start, filtered_rating_start],axis=1)
df_no_na = df[df["Recommended rating"].notna() & df["Filtered rating"].notna()]
average_rating_start = df_no_na["Recommended rating"]
filtered_rating_start = df_no_na["Filtered rating"]
sns.scatterplot(x="Recommended rating", y="Filtered rating",data=df_no_na,alpha=0.5,marker="+")

average_rating_updated = ratings[ratings["Average rating"].notnull()]["Average rating"].rename("Recommended rating")
filtered_rating_updated = yc_updated_reviews[yc_updated_reviews.flagged == "Y"].groupby("businessID").rating.mean().rename("Filtered rating")
sns.scatterplot(x=average_rating_updated, y=filtered_rating_updated,alpha=0.5,marker="+")

x=np.arange(0,6,1)
y1=[1.9588 + 0.5063 * x for x in x]
# y2 = [1.005757 * x for x in x]
sns.lineplot(x=x,y=y1,label="Original data\n(Linear fit $R^2=0.511$)")

x=np.arange(0,6,1)
y1=[1.577581 + 0.588753 * x for x in x]
# y2 = [1.005757 * x for x in x]
fig = sns.lineplot(x=x,y=y1,label="New data\n(Linear fit $R^2=0.572$)")


plt.xlim(1,5)
plt.ylim(1,5)


fig.legend(framealpha=0.25)

plt.show()

fig.get_figure().savefig("../graphs/filtered_vs_rec_rating.png")


display("Original correlation:")
display(scipy.stats.pearsonr(average_rating_start,filtered_rating_start))
display("Updated correlation:")
display(scipy.stats.pearsonr(filtered_rating_updated,average_rating_updated))

In [ ]:
print("New data:")
#Proportional
X = pd.DataFrame(filtered_rating_updated)
y = average_rating_updated
model = sm.OLS(y,X)
results = model.fit()
display(results.params)
display(results.summary())

#Linear
X = pd.DataFrame(filtered_rating_updated)
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
display(results.params)
display(results.summary())

In [ ]:
print("Original data:")
#Proportional
X = pd.DataFrame(filtered_rating_start)
y = average_rating_start
model = sm.OLS(y,X)
results = model.fit()
display(results.params)
display(results.summary())

#Linear
X = pd.DataFrame(filtered_rating_start)
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
display(results.params)
display(results.summary())

# How does the percentage filtered relate to the number of reviews?

In [ ]:
perc_filtered = pd.concat([yc_reviews_we_have.groupby("businessID").apply(lambda df: len(df[df.flagged == "Y"])/len(df)).rename("perc_filtered_o"),
                           yc_updated_reviews.groupby("businessID").apply(lambda df: len(df[df.flagged == "Y"])/len(df)).rename("perc_filtered_n")], axis=1)

In [ ]:
perc_filtered["num_reviews_o"] = yc_reviews_we_have.groupby("businessID").size()
perc_filtered["num_reviews_n"] = yc_updated_reviews.groupby("businessID").size()

In [ ]:
perc_filtered = perc_filtered[perc_filtered.num_reviews_o.notnull()]
perc_filtered_no_outliers = perc_filtered[perc_filtered.num_reviews_o > 15]
perc_filtered_no_outliers = perc_filtered_no_outliers[((perc_filtered_no_outliers.num_reviews_n < 100) | (perc_filtered_no_outliers.perc_filtered_n != 0.0)) & ((perc_filtered_no_outliers.num_reviews_o < 100) | (perc_filtered_no_outliers.perc_filtered_o != 0.0))]

In [ ]:
print("Correlation with outliers (100 or fewer reviews or 0% filtered in either dataset) removed")
print("Correlation reviews, filtered (original): %f (p=%f)" % scipy.stats.pearsonr(perc_filtered_no_outliers.num_reviews_o, perc_filtered_no_outliers.perc_filtered_o))
print("Correlation reviews, filtered (new): %f (p=%f)" % scipy.stats.pearsonr(perc_filtered_no_outliers.num_reviews_n, perc_filtered_no_outliers.perc_filtered_n))
print("Correlation reviews (old), reviews (new): %f (p=%f)" % scipy.stats.pearsonr(perc_filtered_no_outliers.num_reviews_o, perc_filtered_no_outliers.num_reviews_n))
print("Correlation filtered (old), filtered (new): %f (p=%f)" % scipy.stats.pearsonr(perc_filtered_no_outliers.perc_filtered_o, perc_filtered_no_outliers.perc_filtered_n))


In [ ]:
sns.distplot(perc_filtered.perc_filtered_o,label="2012",kde_kws={"clip":(0,1)})
fig = sns.distplot(perc_filtered.perc_filtered_n,label="2020",kde_kws={"clip":(0,1)})
plt.legend()

fig.set_xlabel("Percentage Not Recommended")
fig.set_ylabel("Probability density")

fig.get_figure().savefig("../graphs/filtered_proportion_density.pdf")

print(f"Median: {perc_filtered.perc_filtered_o.median()}, {perc_filtered.perc_filtered_n.median()}")
print(f"KS 2 samp: {scipy.stats.ks_2samp(perc_filtered.perc_filtered_n,perc_filtered.perc_filtered_o)}")

In [ ]:
fig = sns.scatterplot(x="num_reviews_o", y="perc_filtered_o", data=perc_filtered,label="2012", color="steelblue",marker="+")
fig = sns.scatterplot(x="num_reviews_n", y="perc_filtered_n", data=perc_filtered,label="2020", color="coral",alpha=0.5,marker="+")
plt.xscale("log")
fig.set_xlabel("Number of reviews")
fig.set_ylabel("Percentage Not Recommended")
fig.get_figure().savefig("../graphs/filtered_vs_count.pdf")

print(f"Original correlation: {scipy.stats.spearmanr(perc_filtered.num_reviews_o,perc_filtered.perc_filtered_o)}\nNew correlation:{scipy.stats.spearmanr(perc_filtered.num_reviews_n,perc_filtered.perc_filtered_n)}")

In [ ]:
fig = sns.scatterplot(x="num_reviews_n", y="perc_filtered_n", data=perc_filtered,label="New", color="coral")
fig = sns.scatterplot(x="num_reviews_o", y="perc_filtered_o", data=perc_filtered,label="Original", color="steelblue")
fig.get_figure().savefig("../graphs/num_reviews_vs_perc_filtered.png")

In [ ]:

print(perc_filtered[["num_reviews_o","perc_filtered_o"]].sort_values("num_reviews_o"))

In [ ]:
fig = sns.scatterplot(x="perc_filtered_o", y="perc_filtered_n", data=perc_filtered,label="Percent filtered")
fig.get_figure().savefig("../graphs/perc_filtered_original_vs_new.png")

In [ ]:
fig = sns.scatterplot(x="num_reviews_o", y="num_reviews_n", data=perc_filtered,label="Number of reviews")
fig.get_figure().savefig("../graphs/num_reviews_original_vs_new.png")